# Scrapping BasketballReference.com

In [1]:
import MySQLdb as mdb
from bs4 import BeautifulSoup 
import requests
import re
import pandas as pd

In [2]:
# Season Standings 
url_2017standings = "https://www.basketball-reference.com/leagues/NBA_2017.html"

In [3]:
url_2017standings = "https://www.basketball-reference.com/leagues/NBA_2017.html"
standings2017 = requests.get(url_2017standings)
standings2017_soup = BeautifulSoup(standings2017.text, 'html.parser')

## Functions to get regular season conference standings

In [4]:
#Function to scrap conference tables for each year
#Remember to do 2 years back, our data is 2016
def conference_standing(url, number_of_teams, year):
    a = requests.get(url)
    a_soup = BeautifulSoup(a.text, 'html.parser')
    teams = a_soup.findAll('div', class_= 'section_content')[0].findAll('tr', class_='full_table')
    output_list = []
    for i in range(number_of_teams):
        year = year
        team_name = teams[i].find('a').text
        seed = str(teams[i].find('span', class_= 'seed').contents[0])
        seed = re.sub('[()\\xa0]',"",seed)
        wins = teams[i].findAll('td')[0].text
        losses = teams[i].findAll('td')[1].text
        winloss = teams[i].findAll('td')[2].text
        ppg = teams[i].findAll('td')[4].text
        opp_ppg = teams[i].findAll('td')[5].text
        east = 'East' #simple problem, not going to scrape for conference, just use if controls 
        west = 'West'
        if i<15:
            empty_list = [year, team_name, seed, wins, losses, winloss, ppg, opp_ppg, east]
        else:
            empty_list = [year, team_name, seed, wins, losses, winloss, ppg, opp_ppg, west]
        output_list.append(empty_list)
    return output_list

## Functions to get standing before match + starting fives

In [5]:
#SQL query to get dates
con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  database = 'Project',
                  passwd = '<password>', 
                  charset='utf8', use_unicode=True)
    
query_template = '''select count(*), event_date
                    from data_mining.nets_tickets
                    group by event_date
                    '''
cur = con.cursor(mdb.cursors.DictCursor)
cur.execute(query_template, )
data = cur.fetchall()
cur.close()
con.close()
nets_data = pd.DataFrame(list(data))

In [6]:
def reformat_date(event_date_element):
    date1 = str(event_date_element)
    date1 = date1.replace(' 00:00:00', '').replace('-', '')
    date1 = date1 + '0'
    return date1

In [7]:
def home_game_looper(event_date_element):
    
#''' This function uses reformat_date(). Plugs the reformat_date and spits out url''' 

    date = reformat_date(event_date_element)
    url = 'https://www.basketball-reference.com/boxscores/' + date +'BRK.html'
    return url

In [8]:
def current_win_loss(layer1, i):
    
#''' The current_win_loss function inputs layer2, which is the text within table-container and caption, and i,
#which sets to i=0 for wins and i=1 for losses. j=0 for away, j=2 for Nets. It outputs a string'''
    current_standings = re.search('\(.*?\)', layer1).group(0)
    current_standings = re.sub('[\(\)]', '', current_standings)
    current_standings = current_standings.split('-')
    current_standings = [int(i) for i in current_standings]
    return current_standings[i]

In [9]:
def get_startingfive(layer1, i):
    empty_five = []
    for a in range(5):
        if i == 1:
            return "Please choose 2 for Nets"
        else:
            empty_five.append(layer1[i].findAll('th', class_='left')[a].text)
    return empty_five

In [10]:
def teams_info(url):
    
#''' The team_info function uses the current_win_loss function. It takes layer 1, text within table-container,
# and retrieves the wins, losses, team name and starting fivedepending on i= 0 or 1. i=0 for opponent, 
# i=1 for the Nets.  '''

    a = requests.get(url)
    a_soup = BeautifulSoup(a.text, 'html.parser')
    layer1 = a_soup.findAll('div', class_= 'overthrow table_container')
    layer_opp = layer1[0].find('caption').text
    layer_nets = layer1[2].find('caption').text
    opp_wins = current_win_loss(layer_opp, 0) #Set to 0 for win
    opp_losses = current_win_loss(layer_opp, 1) #Set to 1 for win
    nets_wins = current_win_loss(layer_nets, 0)
    nets_losses = current_win_loss(layer_nets, 1)
    opp_name = re.sub('\([^)]*\)', "", layer_opp) #remove everything inside bracket
    opp_name = opp_name.replace("  Table", "")
    nets_five = get_startingfive(layer1, 2)
    opp_five = get_startingfive(layer1, 0)
    output_list = [opp_name, opp_wins, opp_losses, nets_wins, nets_losses]
    output_list = output_list + nets_five #I dont want a list in a list, problematic when encoding to dummy later
    output_list = output_list + opp_five
    return output_list

In [11]:
a = requests.get("https://www.basketball-reference.com/boxscores/201703280BRK.html")
a_soup = BeautifulSoup(a.text, 'html.parser')
layer1 = a_soup.findAll('div', class_= 'overthrow table_container')
layer_opp = layer1[0].find('caption').text

## Function to get a list of all-stars

In [12]:
def find_allstars(url, num_of_players_per_team, year):
    empty_list = []
    a = requests.get(url)
    a_soup = BeautifulSoup(a.text, 'html.parser')
    allstar = a_soup.findAll('div', class_ = 'overthrow table_container')
    for i in range(2, 7): #always going to be a starting 5
        west = allstar[1].findAll('th', class_='left')[i].text
        east = allstar[2].findAll('th', class_='left')[i].text
        empty_list.append([year, west])
        empty_list.append([year, east])
    for j in range(9, num_of_players_per_team + 4): #Extra spaces and labels in between, skip them
        west = allstar[1].findAll('th', class_='left')[j].text
        east = allstar[2].findAll('th', class_='left')[j].text
        empty_list.append([year, west])
        empty_list.append([year, east])
    return empty_list

## Function to get playoff teams

In [13]:
def playoff_teams(url, year):
    empty_list = []
    url = requests.get(url)
    url_soup = BeautifulSoup(url.text, 'html.parser')
    playoffs = url_soup.findAll('div', class_='overthrow table_container')
    champion = url_soup.findAll('div')[0].findAll('p')[2].text.replace('League Champion: ', '')
    num_rounds = len(playoffs[0].findAll('span', class_='tooltip opener'))
    empty_list.append([year, 'Champion', champion])
    for i in range(num_rounds):
        year = year
        rounds = playoffs[0].findAll('span', class_= 'tooltip opener')[i].text #clever trick: 0,1,2
        team1 = playoffs[0].select('a[href*=teams]')[2*i].text #0, 2, 4
        team2 = playoffs[0].select('a[href*=teams]')[2*i+1].text #,1, 3, 5
        team1_list = [year, rounds, team1]
        team2_list = [year, rounds, team2]
        empty_list.append(team1_list)
        empty_list.append(team2_list)
    return empty_list

## Actually Scraping

The functions we will use are **1.** `conference_standing(url, number_of_teams)` **2.** `teams_info(url, i)` **3.** `find_allstars(url, num_of_players_per_team)` **4.** `playoff_teams(url)` and loop **5.** `home_game_looper(event_date_list)`

In [14]:
#Conference standing data
years = ['2017', '2016', '2015', '2014']
conference_data = []

for i in years:
    url = 'https://www.basketball-reference.com/leagues/NBA_' + i + '.html'
    conference_data = conference_data + conference_standing(url, 30, i)

In [15]:
conference_data[0] # looks good

['2017', 'Boston Celtics', '1', '53', '29', '.646', '108.0', '105.4', 'East']

In [16]:
len(conference_data)

120

In [17]:
allstars_data = []

for i in years:
    url = 'https://www.basketball-reference.com/allstar/NBA_' + i + '.html'
    allstars_data.append(find_allstars(url, 12, i))

In [18]:
allstars_data[0][1]

['2017', 'DeMar DeRozan']

In [19]:
#Playoff Appearances + Performance
playoff_data = []

for i in years:
    url = 'https://www.basketball-reference.com/playoffs/NBA_' + i + '.html'
    playoff_data = playoff_data + playoff_teams(url, i)

In [20]:
playoff_data[50] #looks good

['2016', 'Eastern Conference First Round', 'Miami Heat']

## Stats coming into the match will be different, since its indexed by each game not year

In [21]:
def scrap_each_game(event_date):
    each_game = []
    for i in range(3,len(event_date)): #first 3 days have no data
        url = home_game_looper(event_date[i])
        each_game.append([event_date[i],teams_info(url)])
    return each_game

In [22]:
each_game = scrap_each_game(nets_data['event_date'])

In [23]:
each_game[0] #looks good

[Timestamp('2016-10-28 00:00:00'),
 ['Indiana Pacers',
  1,
  1,
  1,
  1,
  'Jeremy Lin',
  'Bojan Bogdanovic',
  'Trevor Booker',
  'Brook Lopez',
  'Rondae Hollis-Jefferson',
  'Paul George',
  'Thaddeus Young',
  'Monta Ellis',
  'Myles Turner',
  'Jeff Teague']]

## Uploading to SQL

## 1. Conference standings

In [71]:
#Connecting to MySQL database
con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  passwd = '<password>', 
                  charset='utf8', use_unicode=True);

In [36]:
# Run a query to create a database that will hold the data
db_name = 'data_mining'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: Warning: (1007, "Can't create database 'data_mining'; database exists")
  import sys


In [25]:
#Create a table for Trending_Descriptions (static data)
cursor = con.cursor()
table_name = 'conference_data'
# Create a table
# The {db} and {table} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (year int,
                                team varchar(250), 
                                seed int,
                                wins int,
                                losses int,
                                winloss decimal(5,3),
                                ppg decimal(5,1),
                                oppg decimal(5,1),
                                conference varchar(250),
                                PRIMARY KEY(year, team)
                                )'''.format(db=db_name, table=table_name)

cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: Warning: (1050, "Table 'conference_data' already exists")


In [30]:
#Creating description table and fetch data 
cursor = con.cursor()
table_name = 'conference_data'

query_template = '''INSERT IGNORE INTO {db}.{table}(year,
                                                    team, 
                                                    seed,
                                                    wins,
                                                    losses,
                                                    winloss,
                                                    ppg,
                                                    oppg,
                                                    conference) 
                                                    VALUES (%s, %s, %s, %s, %s, 
                                                    %s, %s, %s, %s)'''.format(db=db_name, table=table_name)

cursor = con.cursor()

for i in range(len(conference_data)):
    year = int(conference_data[i][0])
    team = conference_data[i][1]
    seed = int(conference_data[i][2])
    wins = int(conference_data[i][3])
    losses = int(conference_data[i][4])
    winloss = float(conference_data[i][5])
    ppg = float(conference_data[i][6])
    oppg = float(conference_data[i][7])
    conference = conference_data[i][8]
    
    query_parameters = (year, team, seed, wins, losses, winloss, ppg, oppg, conference)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

## 2. Playoff Teams

In [66]:
#Connecting to MySQL database
con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  passwd = '<password>', 
                  charset='utf8', use_unicode=True);

In [72]:
#Create a table for Trending_Descriptions (static data)
cursor = con.cursor()
table_name = 'playoff_data'
# Create a table
# The {db} and {table} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (year int,
                                rounds varchar(250), 
                                team varchar(250),
                                PRIMARY KEY(year, team, rounds)
                                )'''.format(db=db_name, table=table_name)

cursor.execute(create_table_query)
cursor.close()

In [73]:
#Creating description table and fetch data 
cursor = con.cursor()
table_name = 'playoff_data'

query_template = '''INSERT IGNORE INTO {db}.{table}(year,
                                                    rounds, 
                                                    team) 
                                                    VALUES (%s, %s, %s)'''.format(db=db_name, table=table_name)

cursor = con.cursor()

for i in range(len(playoff_data)):
    year = int(playoff_data[i][0])
    rounds = playoff_data[i][1]
    team = playoff_data[i][2]
    
    query_parameters = (year, rounds, team)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

## 3. Allstars

In [73]:
#Connecting to MySQL database
con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  passwd = '<password>', 
                  charset='utf8', use_unicode=True);

In [74]:
#Create a table for Trending_Descriptions (static data)
cursor = con.cursor()
table_name = 'allstars_data'
# Create a table
# The {db} and {table} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (year int,
                                player varchar(250), 
                                PRIMARY KEY(year, player)
                                )'''.format(db=db_name, table=table_name)

cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: Warning: (1050, "Table 'allstars_data' already exists")
  if sys.path[0] == '':


In [75]:
#Creating description table and fetch data 
cursor = con.cursor()
table_name = 'allstars_data'

query_template = '''INSERT IGNORE INTO {db}.{table}(year,
                                                    player) 
                                                    VALUES (%s, %s)'''.format(db=db_name, table=table_name)

cursor = con.cursor()

for i in range(len(allstars_data)):
    for j in range(len(allstars_data[i])):
        year = allstars_data[i][j][0]
        player = allstars_data[i][j][1]
        
        query_parameters = (year, player)
        cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1062, "Duplicate entry '2017-Anthony Davis' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1062, "Duplicate entry '2017-DeMar DeRozan' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1062, "Duplicate entry '2017-James Harden' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1062, "Duplicate entry '2017-Giannis Antetokounmpo' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1062, "Duplicate entry '2017-Stephen Curry' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1062, "Duplicate entry '2017-Kyrie Irving' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1062, "Duplicate entry '2017-Kevin Durant' for key 'PRIMARY'")
/usr/local/lib/python3

## 4. Stats coming into match

In [34]:
#Connecting to MySQL database
con = mdb.connect(host = 'localhost', 
                  user = 'root',
                  passwd = '<password>', 
                  charset='utf8', use_unicode=True);

In [37]:
#Create a table for Trending_Descriptions (static data)
cursor = con.cursor()
table_name = 'each_game_data'
# Create a table
# The {db} and {table} are placeholders for the parameters in the format(....) statement
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (event_date datetime,
                                opponent varchar(250), 
                                opp_wins int,
                                opp_losses int,
                                nets_wins int,
                                nets_losses int,
                                nets_1 varchar(250),
                                nets_2 varchar(250),
                                nets_3 varchar(250),
                                nets_4 varchar(250),
                                nets_5 varchar(250),
                                opp_1 varchar(250),
                                opp_2 varchar(250),
                                opp_3 varchar(250),
                                opp_4 varchar(250),
                                opp_5 varchar(250),
                                PRIMARY KEY(event_date)
                                )'''.format(db=db_name, table=table_name)

cursor.execute(create_table_query)
cursor.close()

In [38]:
#Creating description table and fetch data 
cursor = con.cursor()
table_name = 'each_game_data'

query_template = '''INSERT IGNORE INTO {db}.{table}(event_date,
                                                    opponent, 
                                                    opp_wins,
                                                    opp_losses,
                                                    nets_wins,
                                                    nets_losses,
                                                    nets_1,
                                                    nets_2,
                                                    nets_3,
                                                    nets_4,
                                                    nets_5,
                                                    opp_1,
                                                    opp_2,
                                                    opp_3,
                                                    opp_4,
                                                    opp_5)
                                                    VALUES (%s, %s, %s, %s, %s, 
                                                    %s, %s, %s, %s, %s, %s, %s, 
                                                    %s, %s, %s, %s)'''.format(db=db_name, table=table_name)

cursor = con.cursor()

for i in range(len(each_game)):
    event_date = each_game[i][0]
    opponent = each_game[i][1][0]
    opp_wins = each_game[i][1][1]
    opp_losses = each_game[i][1][2]
    nets_wins = each_game[i][1][3]
    nets_losses = each_game[i][1][4]
    nets_1 = each_game[i][1][5]
    nets_2 = each_game[i][1][6]
    nets_3 = each_game[i][1][7]
    nets_4 = each_game[i][1][8]
    nets_5 = each_game[i][1][9]
    opp_1 = each_game[i][1][10]
    opp_2 = each_game[i][1][11]
    opp_3 = each_game[i][1][12]
    opp_4 = each_game[i][1][13]
    opp_5 =  each_game[i][1][14]
    
    query_parameters = (event_date, opponent, opp_wins, opp_losses, nets_wins,
                       nets_losses, nets_1, nets_2, nets_3, nets_4, nets_5,
                       opp_1, opp_2, opp_3, opp_4, opp_5)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()